In [1]:
%run uvms_inverse_dynamics.ipynb

jit after {'jit': True, 'jit_options': {'flags': '-Ofast'}, 'compiler': 'shell'}
number of joints = 4
floating_base found
floating_base found
floating_base found
floating_base found
floating_base found
floating_base found
floating_base found
floating_base found
floating_base found
floating_base found
floating_base found
floating_base found
floating_base found
floating_base found


In [2]:
ptheta = cs.vertcat(base_ss.sim_p, arm_ss.sim_p)

In [3]:
UVMS_ID_coupled = UVMS_ID_use_coupled(use_coupled, states,  uvms_ss.uvms_acc, vehicle_p, manipulator_p, base_T)

# coupled uvms regressor matrix instance
ID_coupled_regressor = cs.jacobian(UVMS_ID_coupled, ptheta)
print(ID_coupled_regressor.size())

UVMS_regressor_coupled_eval = cs.Function('UVMS_regressor_coupled', [states,  uvms_ss.uvms_acc, vehicle_p, manipulator_p, base_T], [ID_coupled_regressor])

(10, 65)


In [4]:
UVMS_ID_decoupled = UVMS_ID_use_coupled(not use_coupled, states,  uvms_ss.uvms_acc, vehicle_p, manipulator_p, base_T)

# decoupled uvms regressor matrix instance
# useful for system identification of standalone manipulator or standalone vehicle base
ID_decoupled_regressor = cs.jacobian(UVMS_ID_decoupled, ptheta)
print(ID_decoupled_regressor.size())

UVMS_regressor_decoupled_eval = cs.Function('UVMS_regressor_decoupled', [states,  uvms_ss.uvms_acc, vehicle_p, manipulator_p, base_T], [ID_decoupled_regressor])

(10, 65)


In [5]:
# conditional inertia matrix instance
UVMS_regressor_use_coupled = cs.Function.if_else('UVMS_regressor_use_coupled', UVMS_regressor_coupled_eval, UVMS_regressor_decoupled_eval)

In [6]:
UVMS_regressor_use_coupled.save("UVMS_regressor_use_coupled.casadi")

In [23]:
x0 = np.array([0, 0, 0.1, 0, 0, 0, 0.0, 0.0, 1.0, 0.0, 1, 2, 0, 1, 1, 1, 5, 1, 2, 1]) #x y z r p y q0 q1 q2 q3 u v w p q r qd0 qd1 qd2 qd3 qd4
a0 = [0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0]

# sparse regressor output
UVMS_regressor_use_coupled(use_coupled, x0, a0, blue.sim_p, alpha.sim_p, alpha.base_T0)@cs.vertcat(blue.sim_p, alpha.sim_p)

DM([-25.73, 116.54, 32.465, 1.16, 0.7, 16.02, 15, 2.29998, 4.40001, 0.299997])

In [24]:
UVMS_ID_use_coupled(use_coupled, x0, a0, blue.sim_p, alpha.sim_p, alpha.base_T0)

DM([-26.4951, 116.487, 33.2231, 1.37349, 1.0167, 16.0659, 15, 2.30002, 4.39999, 0.300003])